Referensi : https://huggingface.co/docs/transformers/tasks/sequence_classification

Import Library yang dibutuhkan

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig, TextClassificationPipeline
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import gc

Jangan lupa gunakan GPU untuk melakukan training

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

Model yang menjadi bahan finetuning adalah `distilbert-base-multilingual-cased` yang merupakan model DistilBERT yang sudah ditraining dengan banyak bahasa, termasuk bahasa Indonesia.

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=3)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'classifie

Kita menggunakan Dataset IndoNLU sebagai dataset fine-tuning

In [4]:
df_train = pd.read_csv('../dataset/smsa_doc-sentiment-prosa/train_preprocess.tsv', sep='\t', header=None)
df_test = pd.read_csv('../dataset/smsa_doc-sentiment-prosa/test_preprocess.tsv', sep='\t', header=None)

df_train.columns = ['text', 'label']
df_test.columns = ['text', 'label']

In [5]:
df_train.sample(3)

,text,label
9887,lokasi di atas puncak pegunungan dengan pemand...,positive
6902,tempat nya kecil dan sempit tapi cukup nyaman ...,positive
270,saya suka makan di sini kalau ke bandung . mak...,positive


Jika token lebih dari 512, kita harus memotongnya

In [6]:
max_length = 512

In [27]:
def sentiment_to_label(sentiment):
    if sentiment == 'positive':
        return 1
    elif sentiment == 'negative':
        return 0
    else:
        return 2

In [8]:
class SentimentDataset(Dataset):
    def __init__(self, df,tokenizer):
        self.encodings = tokenizer(df["text"].tolist(), truncation=True, padding=True)
        self.labels = df["label"].apply(sentiment_to_label).values
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [9]:
train_dataset = SentimentDataset(df_train, tokenizer)
test_dataset = SentimentDataset(df_test, tokenizer)

In [10]:
gc.collect()
torch.cuda.empty_cache()

Atur hyperparameter agar model yang kita gunakan mendapatkan performa maksimal

In [16]:
training_args = TrainingArguments(
    output_dir='../model/smsa-distilbert-indo/',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,
    fp16=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Lakukan Training pada data yang sudah kita siapkan

In [17]:
trainer = Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
                eval_dataset=test_dataset, tokenizer=tokenizer)

Using amp half precision backend


In [18]:
trainer.train(resume_from_checkpoint=True)

Loading model from ../model/smsa-distilbert-indo/checkpoint-2000).
/home/karuniaperjuangan/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3440
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 2000
  Will skip the first 2 epochs then the first 624 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the trai

  0%|          | 0/624 [00:00<?, ?it/s]

Step,Training Loss
2200,0.099400
2400,0.099100
2600,0.067600
2800,0.075100
3000,0.049400
3200,0.041800
3400,0.039800


Saving model checkpoint to ../model/smsa-distilbert-indo/checkpoint-2500
Configuration saved in ../model/smsa-distilbert-indo/checkpoint-2500/config.json
Model weights saved in ../model/smsa-distilbert-indo/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in ../model/smsa-distilbert-indo/checkpoint-2500/tokenizer_config.json
Special tokens file saved in ../model/smsa-distilbert-indo/checkpoint-2500/special_tokens_map.json
Saving model checkpoint to ../model/smsa-distilbert-indo/checkpoint-3000
Configuration saved in ../model/smsa-distilbert-indo/checkpoint-3000/config.json
Model weights saved in ../model/smsa-distilbert-indo/checkpoint-3000/pytorch_model.bin
tokenizer config file saved in ../model/smsa-distilbert-indo/checkpoint-3000/tokenizer_config.json
Special tokens file saved in ../model/smsa-distilbert-indo/checkpoint-3000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3440, training_loss=0.027769904219826988, metrics={'train_runtime': 214.9907, 'train_samples_per_second': 255.825, 'train_steps_per_second': 16.001, 'total_flos': 2504506419360000.0, 'train_loss': 0.027769904219826988, 'epoch': 5.0})

Jangan lupa simpan model dan tokenizer

In [20]:
model.save_pretrained('../model/smsa-distilbert-indo/')
tokenizer.save_pretrained('../model/smsa-distilbert-indo/')

Configuration saved in ../model/smsa-distilbert-indo/config.json
Model weights saved in ../model/smsa-distilbert-indo/pytorch_model.bin
tokenizer config file saved in ../model/smsa-distilbert-indo/tokenizer_config.json
Special tokens file saved in ../model/smsa-distilbert-indo/special_tokens_map.json


('../model/smsa-distilbert-indo/tokenizer_config.json',
 '../model/smsa-distilbert-indo/special_tokens_map.json',
 '../model/smsa-distilbert-indo/vocab.txt',
 '../model/smsa-distilbert-indo/added_tokens.json',
 '../model/smsa-distilbert-indo/tokenizer.json')

In [14]:
config = AutoConfig.from_pretrained('../model/smsa-distilbert-indo/', label2id={"negative":0,"positive":1,"neutral":2}, id2label={0:"negative",1:"positive",2:"neutral"})
model = AutoModelForSequenceClassification.from_pretrained('../model/smsa-distilbert-indo/', config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained('../model/smsa-distilbert-indo/')
model.save_pretrained('../model/smsa-distilbert-indo/')

# Tes Predict

In [28]:
predict = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

In [37]:
predict("Produk ini dirilis kemarin")

[{'label': 'neutral', 'score': 0.9974834322929382}]